# Actividad

Cree una función **features_pipeline** que reciba la imagen original segmentada del taller de repaso y su máscara, a partir de estas calcule los descriptores de forma vistos anteriormente (basta con momentos de Hu) y los guarde en un diccionario junto con los descriptores de color y textura vistos en la sección anterior.

In [1]:
# Importamos todas las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
from skimage.measure import label, regionprops
from scipy.stats import kurtosis, skew
import pandas as pd

In [2]:
def get_momentsHu(mask_region):
    dict_hu = {}
    for i,moment in enumerate(mask_region.moments_hu):
        # Redondeo a 3 decimales
        dict_hu['hu'+str(i)] = round(-1 * math.copysign(1, moment) * math.log10(abs(moment)), 3)
    return dict_hu

In [3]:
def get_first_order_stats(img, mask_region):
    dict_stats = {}
    r = img[:,:,0]
    g = img[:,:,1]
    b = img[:,:,2]
    r_1 = []
    g_1 = []
    b_1 = []
    # Recorrer las posiciones 'i,j' de la matriz de la máscara
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Condicional para que solo los pixeles en las coordenas 'i,j' de la máscara sean añadidos
            if( (mask_region[i][j] == np.array([True, True, True])).all() ):
                r_1.append(r[i][j])
                g_1.append(g[i][j])
                b_1.append(b[i][j])

    channels = [r_1, g_1, b_1]
    channel_symbol = ['_r', '_g', '_b']
    for i,img_channel in enumerate(channels):
        
        # Redondeo a 3 decimales
        dict_stats['mean' + channel_symbol[i]] = round(np.mean(img_channel), 3)
        dict_stats['std' + channel_symbol[i]] = round(np.std(img_channel), 3)
        dict_stats['kurtosis' + channel_symbol[i]] = round(kurtosis(img_channel), 3)
        dict_stats['skewness' + channel_symbol[i]] = round(skew(img_channel), 3)
    
    return dict_stats


In [4]:
def features_pipeline(img_segm, mask, features_list):
    img_labels = label(mask)
    # Cuento cuantos objetos hay (este método no incluye el fondo)
    num_regions = len(regionprops(img_labels))

    # El rango comienza desde 1 pues, la funcion 'label' pone SIEMPRE en la primera etiqueta el fondo
    for i in range(1,num_regions+1):
        # Obtenemos la máscara con solo el i-ésimo objeto detectado por 'label' originalmente
        mask_input = (img_labels==i)*mask
        moments_input = regionprops(label(mask_input))

        # Como se hace 'regionprops' de un solo objeto, solo se toma el primer elemento
        charecteristics_dict = get_momentsHu(moments_input[0])
        
        # Sacamos ahora el objeto/cuerpo segmentado a color
        img_segm_input = cv2.bitwise_and(img_segm, img_segm, mask = mask_input.astype(np.uint8))

        # Calculamos estadísticas de primer orden
        charecteristics_dict.update(get_first_order_stats(img_segm_input, mask_input))

        features_list.append(charecteristics_dict)

In [5]:
img_wheel = cv2.cvtColor(cv2.imread('Repaso/llanta_segmentada.jpg'), cv2.COLOR_BGR2RGB)
# Es necesario garantizar la binarización
mask_wheel = cv2.imread('Repaso/llanta_mask.jpg', cv2.IMREAD_GRAYSCALE) > 127
features_total = []
features_pipeline(img_wheel, mask_wheel, features_total)
for features_set in features_total: 
    print(features_set)

{'hu0': 0.657, 'hu1': 1.647, 'hu2': 4.337, 'hu3': 5.43, 'hu4': -10.321, 'hu5': -6.26, 'hu6': -11.047, 'mean_r': 39.447, 'std_r': 22.483, 'kurtosis_r': 2.442, 'skewness_r': 1.136, 'mean_g': 51.407, 'std_g': 22.388, 'kurtosis_g': 1.664, 'skewness_g': 0.713, 'mean_b': 67.063, 'std_b': 23.992, 'kurtosis_b': 0.836, 'skewness_b': 0.266}
{'hu0': 0.611, 'hu1': 1.468, 'hu2': 5.89, 'hu3': 6.636, 'hu4': 13.002, 'hu5': 7.62, 'hu6': -13.111, 'mean_r': 50.725, 'std_r': 27.494, 'kurtosis_r': 0.749, 'skewness_r': 0.788, 'mean_g': 63.703, 'std_g': 27.631, 'kurtosis_g': 0.421, 'skewness_g': 0.678, 'mean_b': 80.627, 'std_b': 28.513, 'kurtosis_b': 0.041, 'skewness_b': 0.579}


In [6]:
columns = list(features_total[0].keys())
columns.append('class')

# Creamos el dataFrame
df = pd.DataFrame(columns=columns)

# Iteramos sobre la lista con diccionarios de características para cada objeto

for i in range(0,len(features_total)):
    # Se anexa la lista [i] ya que la última columna del dataFrame es 'class'
    df.loc[i] = [features_total[i][feature] for feature in features_total[i].keys()]+[i]

# Exportamos dataFrame a csv
df.to_csv('dataWheels.csv')